#### LeetCode - SQL - #586

Refer [586. Customer Placing the Largest Number of Orders](https://leetcode.com/problems/customer-placing-the-largest-number-of-orders/description/)

Write a solution to find the  `customer_number`  for the customer who has placed  **the largest number of orders**.

The test cases are generated so that  **exactly one customer**  will have placed more orders than any other customer.

In [ ]:
from pyspark import SparkContext

data_orders = [[1, 1], [2, 2], [3, 3], [4, 3]]
columns_orders = ['order_number', 'customer_number']
schema_orders = {'order_number':'Int64', 'customer_number':'Int64'}

In [ ]:
import pandas

pandas_df_orders = pandas.DataFrame(data=data_orders, columns=columns_orders).astype(schema_orders)

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
df_orders = spark_context.createDataFrame(pandas_df_orders)
df_orders.printSchema()
df_orders.show(5)

In [ ]:
from pyspark.sql import functions as F

df_orders_grouped \
    = df_orders.groupBy(df_orders.customer_number).agg(
        F.count(df_orders.order_number).alias("order_count")
    )

df_orders_grouped \
    .orderBy(df_orders_grouped.order_count.desc()) \
    .select(df_orders_grouped.customer_number) \
    .limit(1) \
    .show()

**Follow up:** What if more than one customer has the largest number of orders, can you find all the `customer_number` in this case?

In [ ]:
df_order_count_max = df_orders_grouped.select(F.max(df_orders_grouped.order_count))
order_count_max = df_order_count_max.first()[0]
print(order_count_max)

df_orders_grouped \
    .filter(df_orders_grouped.order_count == order_count_max) \
    .select(df_orders_grouped.customer_number) \
    .show()